### pip install & import

In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd
#import pandas_profiling
import datetime

In [3]:
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import LabelEncoder

from sklearn.preprocessing import MinMaxScaler

from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import GradientBoostingRegressor

from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestRegressor

# from xgboost import XGBRFRegressor
from sklearn.model_selection import cross_validate

### 최종데이터 불러오기

In [4]:
# movie_final_회귀 = pd.read_csv('../3_merge_top300/movie_final_reg.csv',encoding='utf-8')
movie_final_분류 = pd.read_csv('../3_merge_top300/movie_final_cls.csv',encoding='utf-8')

# 스케일링

In [5]:
from sklearn.preprocessing import RobustScaler

# 객체생성
robust_scaler = RobustScaler()

# *** 만약 특정 열의 스케일링을 하고 싶은 경우 ***
A_n = robust_scaler.fit_transform(movie_final_분류['전국 스크린수'].values.reshape(-1,1))
movie_final_분류.insert(0, '전국 스크린수_scaled', A_n)
# movie_final_분류.drop(['전국 스크린수'], axis=1, inplace=False)

A_n1 = robust_scaler.fit_transform(movie_final_분류['총 관객수'].values.reshape(-1,1))
movie_final_분류.insert(0, '총 관객수_scaled', A_n1)
# movie_final_분류.drop(['총 관객수'], axis=1, inplace=False)

In [6]:
movie_final_분류.columns

Index(['총 관객수_scaled', '전국 스크린수_scaled', 'Unnamed: 0', '순번', '영화명', '감독',
       '배급사', '개봉일', '영화형태', '국적', '전국 스크린수', '전국 매출액', '전국 관객수', '서울 매출액',
       '서울 관객수', '장르', '등급', '영화구분', '연도', '월', '계절', '총 관객수', '코로나', '출연',
       '평점', '감독_흥행', '배급사_흥행', '주연배우_흥행'],
      dtype='object')

## 라벨링

In [7]:
from sklearn.preprocessing import LabelEncoder

le_form = LabelEncoder()
le_nation = LabelEncoder()
le_genre = LabelEncoder()
le_rating = LabelEncoder()
le_class = LabelEncoder()
le_season = LabelEncoder()
le_year = LabelEncoder()

movie_labels = movie_final_분류.copy()
movie_labels['영화형태'] = le_form.fit_transform(movie_final_분류['영화형태'])
movie_labels['국적'] = le_nation.fit_transform(movie_final_분류['국적'])
movie_labels['장르'] = le_genre.fit_transform(movie_final_분류['장르'])
movie_labels['등급'] = le_rating.fit_transform(movie_final_분류['등급'])
movie_labels['영화구분'] = le_class.fit_transform(movie_final_분류['영화구분'])
movie_labels['계절'] = le_season.fit_transform(movie_final_분류['계절'])
movie_labels['연도'] = le_year.fit_transform(movie_final_분류['연도'])
movie_labels.drop(['Unnamed: 0'],axis=1,inplace=True)
movie_labels.head(3)


,총 관객수_scaled,전국 스크린수_scaled,순번,영화명,감독,배급사,개봉일,영화형태,국적,전국 스크린수,...,연도,월,계절,총 관객수,코로나,출연,평점,감독_흥행,배급사_흥행,주연배우_흥행
0,0.027876,1.397641,1,명량,김한민,씨제이이엔엠,2014-07-30,0,7,1587,...,2014,7,3,215068958,0,"['최민식', '류승룡', '조진웅']",8.44,9,10,20
1,0.216263,2.056445,2,극한직업,이병헌,씨제이이엔엠,2019-01-23,0,7,1978,...,2019,1,1,226678777,0,"['류승룡', '이하늬', '진선규', '이동휘', '공명']",8.51,4,10,16
2,0.105881,1.945240,3,신과함께-죄와 벌,김용화,롯데엔터테인먼트,2017-12-20,0,7,1912,...,2017,12,1,219876227,0,"['하정우', '차태현', '주지훈', '김향기', '김동욱', '마동석']",7.83,9,9,40


In [ ]:
import numpy as np

movie_labels['분류클래스'] = np.where((movie_labels['전국 관객수'].values<1000000), 0, 
                                 np.where((movie_labels['전국 관객수'].values>1000000) & (movie_labels['전국 관객수'].values<1400000), 1,
                                          np.where((movie_labels['전국 관객수'].values>1400000) & (movie_labels['전국 관객수'].values<2000000), 2,
                                                   np.where((movie_labels['전국 관객수'].values>2000000) & (movie_labels['전국 관객수'].values<3000000), 3,
                                                            np.where((movie_labels['전국 관객수'].values>3000000) & (movie_labels['전국 관객수'].values<5000000), 4, 5))))
)
movie_labels['분류클래스'].unique()

### 변수 정리   
사용하지 않는 컬럼 제거


In [8]:
# 사용하지 않는 컬럼 제거

not_used = ['영화명', '감독', '배급사', '영화형태', '개봉일', '영화구분', '월']

movie_DL = movie_labels.drop(not_used, axis=1)
movie_DL.head()

,총 관객수_scaled,전국 스크린수_scaled,순번,국적,전국 스크린수,전국 매출액,전국 관객수,서울 매출액,서울 관객수,장르,등급,연도,계절,총 관객수,코로나,출연,평점,감독_흥행,배급사_흥행,주연배우_흥행
0,0.027876,1.397641,1,7,1587,135748398910,17613682,"33,121,225,810","4,163,666",9,1,2014,3,215068958,0,"['최민식', '류승룡', '조진웅']",8.44,9,10,20
1,0.216263,2.056445,2,7,1978,139647979516,16264944,"31,858,660,536","3,638,287",16,1,2019,1,226678777,0,"['류승룡', '이하늬', '진선규', '이동휘', '공명']",8.51,4,10,16
2,0.105881,1.945240,3,7,1912,115698654137,14410754,"27,530,825,087","3,346,172",17,0,2017,1,219876227,0,"['하정우', '차태현', '주지훈', '김향기', '김동욱', '마동석']",7.83,9,9,40
3,0.027876,0.351306,4,7,966,110828014630,14245998,"25,842,519,330","3,233,946",4,0,2014,1,215068958,0,"['황정민', '김윤진', '오달수', '정진영']",9.02,10,10,17
4,0.216263,3.500421,5,2,2835,122182694160,13934592,"33,577,136,860","3,597,963",13,0,2019,2,226678777,0,"['로버트 다우니 주니어', '크리스 에반스', '크리스 헴스워스', '마크 러팔로...",9.38,10,10,106


In [9]:
독립 = movie_DL[['장르', '등급', '감독_흥행','총 관객수_scaled',
               '배급사_흥행','주연배우_흥행','계절', '전국 스크린수_scaled','연도','코로나']].copy()

종속 = movie_DL[['분류클래스']].copy()

display(독립.info(), 종속.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 708 entries, 0 to 707
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   장르              708 non-null    int32  
 1   등급              708 non-null    int32  
 2   감독_흥행           708 non-null    int64  
 3   총 관객수_scaled    708 non-null    float64
 4   배급사_흥행          708 non-null    int64  
 5   주연배우_흥행         708 non-null    int64  
 6   계절              708 non-null    int32  
 7   전국 스크린수_scaled  708 non-null    float64
 8   연도              708 non-null    int64  
 9   코로나             708 non-null    int64  
dtypes: float64(2), int32(3), int64(5)
memory usage: 47.1 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 708 entries, 0 to 707
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   전국 관객수  708 non-null    int64
dtypes: int64(1)
memory usage: 5.7 KB


None

None

## train, val, test 데이터 나누기

In [10]:
# 1. train 데이터(80%)와 test 데이터(20%)로 나누기
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(독립, 종속, test_size=0.20)

In [11]:
# 2. train 데이터를 train 데이터(80%*80%)와 val 데이터(80%*20%)로 나누기
train_input, val_input, train_target, val_target = train_test_split(train_input, train_target, test_size=0.20)
print(len(train_input), len(val_input), len(test_input))

452 114 142


# 딥러닝 

In [56]:
# 딥러닝 층 여기가 중요

X = tf.keras.layers.Input(shape=[len(train_input.columns)])

H = tf.keras.layers.Dense(128)(X)
H = tf.keras.layers.BatchNormalization()(H)
H = tf.keras.layers.Activation(tf.nn.swish)(H)

H = tf.keras.layers.Dense(128)(X)
H = tf.keras.layers.BatchNormalization()(H)
H = tf.keras.layers.Activation(tf.nn.swish)(H)

H = tf.keras.layers.Dense(128)(H)
H = tf.keras.layers.BatchNormalization()(H)
H = tf.keras.layers.Activation(tf.nn.swish)(H)
# dropout = tf.keras.layers.Dropout(rate=0.3)(H)

Y = tf.keras.layers.Dense(6, activation='softmax')(H)
model = tf.keras.models.Model(X,Y)
# sgd = tf.keras.optimizers.Adam(learning_rate=0.01,beta_1=0.8,beta_2=0.85)
# sgd = tf.keras.optimizers.SGD(learning_rate=0.1,momentum=0.7)
model.compile(loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [ ]:
# 훈련
history = model.fit(train_input,train_target,batch_size=128,epochs=100,validation_data=(val_input,val_target))

In [ ]:
# 확인용도
model.predict(train_input[:5])

In [ ]:
# 여기랑 비슷한지 보기
train_target[:5]

### test data 확인용도

In [ ]:
model.predict(test_input[:5])

In [ ]:
test_target[:5]